In [4]:
import math

In [242]:
def trial_and_error(n):
    primes = []
    for i in range(2, n+1):
        found = False
        for j in range(2, i):
            found = False
            if i % j == 0:
                found = True
                break
        if not found:
            primes.append(i)
    return primes

In [243]:
trial_and_error(10)

[2, 3, 5, 7]

# Algorithm and code design matters

As your experience in data science grows I have no doubt that one lesson your will repeatedly learn is the importance of a good algorithm.  A good algorithm can turn a seven day model run time into 4 hours (and a 14 day run time due to finding a mistake in the first run into an 8 hour run). A related lesson is that how you design your code to implement an algorithm also matters.  Indeed the decisions you make in implementation can also have an order of magnitude impact on run time.  In this section we will explore a simple problem - computing (small) prime numbers and how the choice of algorithm and its implementation can affect run time.

## Prime numbers

A prime number is an integer greater than 1 that is not a product of two smaller integer values. I.e it can only be divided by itself and one.  As an example, 2 is a prime number, but 4 is not.

## The first $n$ prime numbers

A naive implementation of an algorithm to find the first $n$ prime numbers is **trial division** where each number between 2 and $n$ (actually you only need to go to $\sqrt{n}$) is checked for primality.  This works, but it computationally demanding even for small $n$.

> We won't implement this here as its just too painfully slow!

## A more efficient algorithm: The Sieve of Eratosthenes

A more efficient algorithm design, at least for relatively smally $n$ (e.g. < 1 million), is the **Sieve of Eratosthenes**.  It works as follows:

Consider the primes up to 10. First write out a list of number 2 to 10. Here we'll represent this as a python list called `candidates`.  As we work out which numbers are not primes we will cross (sieve) them out. For simplicity of illustration we will store those in a list called `crossed_out` 

```python
candidates = [2, 3, 4, 5, 6, 7, 8, 9, 10]
crossed_out = []
```
The first candidate number in the list is 2. The Sieve of Eratosthenes algorithm crosses out every 2nd number in the list after 2 by counting up from 2 in increments of 2: 

```python
candidates = [2, 3, 4, 5, 6, 7, 8, 9, 10]
crossed_out = [4, 6, 8, 10]
```
The next number in `candidates` after 2 **that has not been crossed out** is 3. We now cross out every 3rd number in the list by counting up from 3 increments of 3.  

```python
candidates = [2, 3, 4, 5, 6, 7, 8, 9, 10]
crossed_out = [4, 6, 8, 10, 9]
```

This is the basic process that is repeated in each iteration of the Sieve of Eratosthenes.  What might not be obvious is that we do not need to traverse every item in `candidates`, but instead only iterate up to the $\sqrt{n}$.  At the end of the algorithm the numbers that have not been crossed out are the prime numbers between 2 and $n$  

```python
primes = [i for i in candidates if i not in crossed_off]  
```

Let's implement the sieve in standard python.  Our first implementation will be fairly naive. We will look at its inefficiencies and think through how we can improve on them.  We'll need some test data first.  The dict `n_primes` holds the number of primes up to a value of n increasing by a factor of 10 each time.  For example, there are 25 primes numbers in the first 100 natural numbers.

In [11]:
n_primes = {10 : 4,                 
            100 : 25,               
            1000 : 168,
            10000 : 1229,
            100000 : 9592,
            1000000 : 78498,
            10000000 : 664579}

## Implementation 1.

The function below `prime_sieve` is a naive implementation of this algorithm.  It makes use of two list data structures that we used above when working through the sieve's logic. 
* `candidates`: a simple sequence of numbers 0 to `n`; 
* `crossed_out` a list of numbers sieved.  

Although a naive implementation its not a bad first draft of a function: its readable and works. 

The implementation and test below provides expected results and for it takes around 10ms (at least on my machine) to compute all the prime numbers up to 1000.  

In [201]:
def prime_sieve(n):
    '''
    Compute the prime numbers between 1 and n.
    Naive implementation of the Sieve of Eratosthenes.
    
    Parameters:
    ----------
    n: int
        The upper limit
    
    Returns:
    -------
    list
        a list of primes
    '''
    # a list of candidate numbers to sieve
    candidates = [i for i in range(n + 1)]
    # a list of numbered eliminated from consideration
    crossed_out = [0, 1]
    # maximum iterations required
    limit = int(math.sqrt(n)) + 1
    
    for i in range(2, limit):
         for j in range(i+i, n+1, i):              
                if not candidates[j] in crossed_out:
                    crossed_out.append(candidates[j])
    
    return [i for i in candidates if i not in crossed_out]   

In [202]:
prime_sieve(10)

[2, 3, 5, 7]

In [203]:
print(len(prime_sieve(100)) == n_primes[100])
print(len(prime_sieve(1_000)) == n_primes[1_000])

True
True


In [204]:
%timeit (prime_sieve(1_000))

13.2 ms ± 67.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## One data structure instead of two.

An inefficiency with our code is that we compute the prime numbers using two data structures.  This wastes memory, but also means we need to check if a values already exists in `crossed_out` twice.  The second check is needed to make sure we aren't needlessly appending an already crossed out prime number.  

The function `prime_sieve2` delivers the same results as `prime_sieve`, but more efficiently.  The primary source of this saving is by designing our code so that we required only one `list`.

In [190]:
def prime_sieve2(n):
    # We will use boolean's here instead of ints
    candidates = [True] * (n + 1)
    candidates[0] = candidates[1] = False
    limit = int(math.sqrt(n)) + 1    
    
    for i in range(2, limit): 
        if candidates[i]:
            candidates[i+i::i] = [False] * ((n - i) // i)
            
    return [i for i in range(n+1) if candidates[i]]        

In [164]:
# print(len(prime_sieve2(100)) == n_primes[100])
print(len(prime_sieve2(1000)) == n_primes[1000])

False
False


In [174]:
%timeit prime_sieve2(1000)

36.7 µs ± 282 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Does the time saving matter?

You should see a substantial reduction in computational overhead of `prime_sieve2`.  We are now finding the primes under 1000 in microseconds (µs) as opposed milliseconds in our naive implementation (1 µs = 1000ms!). 

Some of you might be thinking "*both procedures give the same results and execution takes under a second. So does this type of optimisation matter?*" To answer that question let's compute primes under 10,000.  You should now see a much bigger difference with our naive function returning in around a second while `prime_sieve2` is still in µs (on my machine still below half a millsecond).

> When first writing this section I first tested `prime_sieve` with `n=1_000_000`. Needless to say, I quickly decided to use an example of 10,000 instead!   When you fancy a cup of tea or coffee set `n=1_000_000` and come back after your break to see an enourmous difference in performance!

In [120]:
TEN_THOUSAND = 10_000
%timeit len(prime_sieve(TEN_THOUSAND))

1.09 s ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [175]:
%timeit prime_sieve2(TEN_THOUSAND)

355 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Taking a look at the redesign

The redesigned implementation only has single list `candidates`: its contains only `bool` values.  In python that won't make much, if any, difference to memory usage, as `bool` is really a special type of int.  But its a nice simplification as we the index of the list item gives us our factor.  This modification provides our biggest speed up.  Each time we 'cross out' a number e.g. `i` we simply set `candidate[i] = False`. (We don't bother checking as we are not appending to a second list). At the end of the function any items that are still `True` are the primes.

> Don't believe me about `int` and `bool`?  Try executing `(True + 4)`.

The second change might be considered a 'micro optimisation'. I.e. list slicing in favour of an inner `for` loop.

```python
candidates[i+i::i] = [False] * ((n - i) // i)
```

Here we start at the current index + one factor ahead e.g. if `i=2` then we start at index `i+i=4` and select items in steps of `i=2` all the way to the end `candidates[i+i::i]` (Note we could also have used `candidates[i+i:n+1:i]`).  In general, a slice will be faster than looping over the list.  Here's a simpler example of list slicing with a step:


In [151]:
list_to_slice = [i for i in range(31)]
list_to_slice[4:len(list_to_slice):2]

[4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]

## Do micro-optimisations matter?

In short micro-optimisations of your python code matter less than an efficient design.  I.e. 'trial and error' versus a 'prime sieve' or multiple versus single data structures (at least in this example) is more important than a for loop versus a list slice.  The difference is that the former gave us an order of magnitude speed up while the latter helped, but to a much smaller extent.

Another example of a **micro-optimisation** (at least in standard python) relates to the type of data structure used.  `prime_sieve2` made use of a list of boolean values.  Let's have a look at its size in memory in bytes.

In [230]:
import sys
candidates_bool = [True] * (10)
sys.getsizeof(candidates_bool)

136

As we only effectively track 0's (False) and 1's (True) there is an option to shrink the memory requirement substantially by using a python `bytearray`.

In [236]:
candidates_bytes =  bytearray(b"\x01") * 10
print(candidates_bytes)
print(sys.getsizeof(candidates_bytes))

bytearray(b'\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01')
67


That approximately halves memory usage.  That's a nice feature.  `prime_sieve3` updates our sieve to use a `byte_array`.  We will also compare its performance to `prime_sieve2`

In [221]:
def prime_sieve3(n):
    # bytearray here instead of bools to reduce memory overhead.
    candidates = bytearray(b"\x01") * (n + 1)
    # 0 and 1's instead of False and True
    candidates[0] = 0
    candidates[1] = 0
    limit = int(math.sqrt(n)) + 1    
    
    for i in range(2, limit): 
        if candidates[i]:
            candidates[i+i::i] = [0] * ((n - i) // i)
            
    return [i for i in range(n+1) if candidates[i]]      

In [234]:
%timeit prime_sieve3(TEN_THOUSAND)

433 µs ± 4.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Well that's a bit of a surprise.  Our memory efficient `prime_sieve3` is **slightly** slower on average than `prime_sieve2`.  But, and its a big BUT, that's only for the first 10,000 natural numbers.  When we compute larger primes the pattern reverses. For example, `prime_sieve3` is faster to compute the primes up to 10 million.  This performance gap widens as we compute larger and larger primes.  So for very small primes this 'optimisation' doesn't help, but it may help for larger primes (large is of course relative).   

In [237]:
TEN_MILLION = 10_000_000
%timeit prime_sieve2(TEN_MILLION)
%timeit prime_sieve3(TEN_MILLION)

560 ms ± 6.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
467 ms ± 2.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## What else might you do?

We chose a particular algorithm to improve on trial and error in the computation of primes.  So the big questions are


* are there other algorithms and options that might reduce runtime?   
* is this good performance good enough or a problem for our study?
* are we missing any obvious big ticket design changes?

There are in fact other algorithms which are more efficient at computing larger primes than our sieve, but our sieve is reasonably good for the small primes we have computed.  Could we make further improvements in its basic design? Yes we can.  For instance.  We know that all even numbers above two cannot be primes.  We therefore don't even need to consider these in our algorithm.  This not only reduces computation, but we can also half the size of our data structure (you might try this an exercise).

Another option that we will explore in a later section is `numpy`.  This provides some very fast optimised data structures and procedures that we will compare to standard python.

## Summing up

We've seen that a good algorithm makes a huge difference to what is feasible to compute.  A trial and error approach to computing primes prevented us from even finding relatively small prime numbers in a reasonable time frame.  The **Sieve of Eratosthenes** made the unfeasible suddenly feasible.  We've also seen that the design of code can also effect execution time by an order of magnitude.  But not all optimisations will have a huge impact on performance and some optimisation may scale with our problem size.  